In [4]:
import sys
sys.path.append('./models')
sys.path.append('./helpers')

In [5]:
from models.bm25_base import BM25_json

In [3]:
BM25_json('London')

({1: {'rank': 1,
   'doc_id': 7335,
   'score': 0.0022875937685103175,
   'url': 'https://www.bbc.com/news/uk-england-london-64798395',
   'header': 'ULEZ expansion: Contested claims examined',
   'text': "ULEZ expansion: Contested claims examined The Ultra Low Emission Zone (ULEZ) is expanding to cover the whole of London from the end of August, much to the dismay of the opponents of the scheme. The BBC has been assessing some of the claims made about London Mayor Sadiq Khan's flagship clean-air policy to better understand its impact. Opposition from Liberal Democrats and some Labour councils and MPs focuses on the argument that people need more time to swap their vehicles for compliant ones.  The mayor of London announced his ULEZ expansion plan at the end of November, giving people nine month",
   'session_id': 4},
  2: {'rank': 2,
   'doc_id': 7186,
   'score': 0.0022426591076623193,
   'url': 'https://www.bbc.com/news/england/london',
   'header': 'BBC News London',
   'text': "BB

In [6]:
from models.desm import DESM_json

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [23]:
score = DESM_json('david attenborough')[1]

In [15]:
BM25_json('attenborough')[1]

[0.01629995338427853,
 0.015568552454888713,
 0.013871295096971964,
 0.013749142969198234,
 0.013629123431354634,
 0.013494498646204493,
 0.01332991207555304,
 0.01329747536299783,
 0.01328131613286507,
 0.013121857952544192]

In [17]:
import numpy as np

In [61]:
from sklearn.metrics import ndcg_score
true_relevance = np.asarray([[2, 0, 1, 0, 1,0, 0, 3, 0,0]])
scores = np.asarray([score])

ndcg_score(true_relevance,scores)


0.7382225928616724

In [57]:
scores*2

array([[0.0032281 , 0.00319413, 0.00317978, 0.00315533, 0.00312842,
        0.00312631, 0.00312338, 0.00312211, 0.00312136, 0.00311338]])

1.0